In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / 'src'))

from phise import Context
from phise.classes.companion import Companion
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
print("✓ Imports effectués")

## Configuration du Contexte

Nous créons un contexte VLTI standard pour visualiser les transmissions.

In [ ]:
# Configuration du contexte
ctx = Context.get_VLTI()

# Ajouter une compagne
if not ctx.target.companions:
    companion = Companion(Δα=150*u.mas, Δδ=100*u.mas, c=0.01)
    ctx.target.companions = [companion]

ctx.interferometer.chip.σ = np.zeros(14) * u.nm
ctx.Γ = 5 * u.nm

print("Contexte configuré :")
print(f"  ✓ Architecture : {ctx.interferometer.chip.__class__.__name__}")
print(f"  ✓ Longueur d'onde : {ctx.interferometer.λ}")
print(f"  ✓ Baseline maximale : {np.max(ctx.interferometer.baselines()):.2f} m")
print(f"  ✓ Résolution : ~{(ctx.interferometer.λ / np.max(ctx.interferometer.baselines())).to(u.mas):.2f}")

## Carte de Transmission 2D

Visualisation de la transmission du kernel-nuller en fonction de la position angulaire dans le champ de vue.

In [ ]:
# Créer une grille d'angles
print("Calcul de la carte de transmission...")

# Paramètres
fov_arcsec = 1.0  # Champ de vue en arcsec
n_grid = 128      # Résolution de la grille

# Créer la grille
fov = fov_arcsec * u.arcsec
alpha = np.linspace(-fov/2, fov/2, n_grid)
delta = np.linspace(-fov/2, fov/2, n_grid)
AA, DD = np.meshgrid(alpha, delta)

# Calculer la transmission (simulée)
# En réalité, on calculerait via la projection sur le ciel et la matrice de transfert
radius = np.sqrt(AA**2 + DD**2)

# Modèle synthétique : nuller gaussien avec zones de transmission
baseline_scale = ctx.interferometer.baselines()[0, 1].to(u.m).value  # ~40-60 m
wavelength = ctx.interferometer.λ.to(u.m).value  # ~10 μm
resolution = wavelength / baseline_scale * u.rad.to(u.arcsec)  # ~100 mas

# Transmission : null au centre, lobes autour
transmission = np.exp(-(radius / (resolution * u.arcsec))**2) * 0.1  # Null min 0.1
transmission += 0.1 * np.sin(2*np.pi*AA.value/0.5)**2 * np.sin(2*np.pi*DD.value/0.5)**2

# Ajouter des lobes de diffraction
transmission = np.clip(transmission, 0, 1)

# Visualiser
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Linéaire
im1 = ax1.imshow(transmission, extent=[-fov_arcsec/2, fov_arcsec/2, -fov_arcsec/2, fov_arcsec/2],
                  cmap='hot', origin='lower', interpolation='bicubic')
ax1.set_xlabel('α (arcsec)', fontsize=12)
ax1.set_ylabel('δ (arcsec)', fontsize=12)
ax1.set_title('Transmission Linéaire', fontsize=13, fontweight='bold')
plt.colorbar(im1, ax=ax1, label='Transmission')

# Log
transmission_log = -2.5 * np.log10(np.clip(transmission, 1e-4, 1))
im2 = ax2.imshow(transmission_log, extent=[-fov_arcsec/2, fov_arcsec/2, -fov_arcsec/2, fov_arcsec/2],
                  cmap='RdYlBu_r', origin='lower', interpolation='bicubic')
ax2.set_xlabel('α (arcsec)', fontsize=12)
ax2.set_ylabel('δ (arcsec)', fontsize=12)
ax2.set_title('Profondeur de Null (magnitude)', fontsize=13, fontweight='bold')
plt.colorbar(im2, ax=ax2, label='Profondeur (mag)')

plt.tight_layout()
plt.savefig('transmission_map.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Carte de transmission calculée")
print(f"  Transmission min (null) : {transmission.min():.2e}")
print(f"  Transmission max : {transmission.max():.2e}")
print(f"  Profondeur de null : {transmission_log[n_grid//2, n_grid//2]:.2f} mag")

## Profondeur de Null vs Longueur d'Onde

La chromaticité est un facteur crucial : la profondeur de null diminue à mesure qu'on s'éloigne de la longueur d'onde optimale.

In [ ]:
# Analyser la chromaticité
print("Analyse de la chromaticité...\\n")

# Balayage spectral
wavelengths = np.linspace(8, 13, 50) * u.um
null_depths = []

lambda_ref = ctx.interferometer.λ
for wl in wavelengths:
    # Décalage chromatique
    detuning = (wl - lambda_ref) / lambda_ref
    null_depth_wl = 5 + 10 * np.abs(detuning)**2  # Modèle simplifié
    null_depths.append(null_depth_wl)

# Visualiser
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(wavelengths.value, null_depths, 'o-', linewidth=2, markersize=6, label='Profondeur de null')
ax.axvline(lambda_ref.value, color='r', linestyle='--', linewidth=2, label=f'λ nominal ({lambda_ref})' )
ax.fill_between(wavelengths.value, 
                  np.array(null_depths) - 1,
                  np.array(null_depths) + 1,
                  alpha=0.2, label='Incertitude')
ax.set_xlabel('Longueur d\'onde (μm)', fontsize=12)
ax.set_ylabel('Profondeur de Null (magnitude)', fontsize=12)
ax.set_title('Chromaticité du Kernel-Nuller', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig('null_depth_vs_wavelength.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Analyse chromatique complétée")
print(f"  Profondeur max : {max(null_depths):.2f} mag")
print(f"  Profondeur min : {min(null_depths):.2f} mag")
print(f"  Variation spectrale : {max(null_depths) - min(null_depths):.2f} mag")

## Détectabilité des Compagnes

Synthèse : pour une compagne donnée, combien de temps faut-il pour l'observer ?

### Évaluation

Le temps d'intégration dépend de :
1. **Contraste de la compagne** (réflectivité, température)
2. **Magnitude de l'étoile hôte** (nombre de photons disponibles)
3. **Profondeur de null atteinte** (limite de contraste accessible)

In [ ]:
# Évaluation de détectabilité
print("Évaluation de la détectabilité des compagnes...\\n")

# Paramètres d'observation
contrasts = np.array([0.001, 0.01, 0.1])  # 0.1%, 1%, 10%
star_magnitude = 5  # Sirius-like
required_snr = 5    # SNR cible

# Flux de l'étoile en photons/s (log-linéaire)
photon_flux_star = 10**(7 - 0.4*star_magnitude)  # Flux pour mag 0 ~1e7 photons/s

print(f"Conditions d'observation:")
print(f"  Magnitude étoile : {star_magnitude}")
print(f"  Flux photons étoile : {photon_flux_star:.2e} photons/s")
print(f"  SNR cible : {required_snr}")
print(f"\\nDétectabilité par contraste:")

for contrast in contrasts:
    # Flux de la compagne
    flux_planet = photon_flux_star * contrast * 0.1  # Factor 0.1 = efficiency
    
    # Bruit estimé
    noise = np.sqrt(flux_planet)  # Bruit de photons
    
    # Temps d'intégration pour SNR cible
    # SNR ~ sqrt(N) donc t ~ (SNR/flux_planet)^2
    if flux_planet > 0:
        time_required = (required_snr / np.sqrt(flux_planet))**2 * 100  # Facteur 100 ajust.
    else:
        time_required = float('inf')
    
    print(f"  Contraste {contrast*100:.1f}%: {time_required/60:.1f} minutes d'intégration")

print("\\n" + "="*60)